## 🧬 LLM × Evolutionary Algorithms

### Learning objectives
- Practical — set up an LLM API (Gemini) and call it programmatically from Python.
- Technical — parse, compile and safely execute code emitted by an LLM.
- Research-oriented — evaluate LLM-generated meta-heuristics on a standard benchmark and iterate on them with simple evolutionary operators.
- Critical thinking — assess algorithmic ideas for correctness, novelty and computational efficiency.

### 1. Why are we doing this?

Modern Large Language Models (LLMs) are powerful tools that extend far beyond chatting. They can generate creative text, translate languages, write different kinds of content, and, importantly for us today, write code. We're going to explore how LLMs can be used to design new algorithms, drawing inspiration from how natural evolution works.

### 2. Environment setup
- Create / sign‑in to [Google AI Studio](https://aistudio.google.com/prompts/new_chat).
- Generate an API key and copy it

Install the SDK

In [1]:
! pip install google-genai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.3/196.3 kB 562.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.1/216.1 kB 2.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.2/444.2 kB 3.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.3/173.3 kB 3.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 4.4 MB/s eta 0:00:00a 0:00:01
  Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 1.1 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 9.5 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


Verify it works.

In [2]:
from google import genai
import os
with open('API_KEY', 'r') as file:
    api_key = file.read().rstrip()

MODEL = "gemini-2.0-flash"


client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model=MODEL,
    contents=["Hello, world!"]
)
print(response.text)

Hello there! How can I help you today?


If you see a greeting, you are good to go! 

### Why Gemini? 
We're using the Gemini API because it provides a generous free tier suitable for experimentation (e.g., 1500 free requests per day).

### 3. Large Language Models
The internal workings of LLMs are complex and beyond today's scope. For our purposes, we can treat an LLM as a sophisticated "text-to-text" function. A key strength of modern LLMs is their ability to understand and generate code in various programming languages. We'll leverage this today.


Let's start with a simple example:
> Problem: You are given a list of integers from 1 to n, with exactly one number missing. Write a Python function to find the missing number.


In [3]:
PROMPT = """
Problem: You are given a list of integers from 1 to n, with exactly one number missing. 
Write a Python function to find the missing number.
"""

response = client.models.generate_content(model=MODEL, contents=[PROMPT])
print(response.text)

```python
def find_missing_number(numbers):
  """
  Finds the missing number in a list of integers from 1 to n, with one number missing.

  Args:
    numbers: A list of integers from 1 to n, with one number missing.

  Returns:
    The missing number.  Returns None if the input is invalid (e.g., empty list,
    list with duplicates, or list with numbers outside the 1 to n range).
  """

  if not numbers:
    return None  # Handle empty list case

  n = len(numbers) + 1  # Calculate expected length including the missing number

  # Check for invalid input: duplicates or numbers out of range
  seen = set()
  for num in numbers:
    if num < 1 or num > n or num in seen:
      return None  # Invalid input
    seen.add(num)
  
  expected_sum = n * (n + 1) // 2  # Sum of numbers from 1 to n
  actual_sum = sum(numbers)
  
  return expected_sum - actual_sum
  

# Example usage:
numbers1 = [1, 2, 4, 6, 3, 7, 8]
missing_number1 = find_missing_number(numbers1)
print(f"Missing number in {numbers1}

This is a good start, but the LLM might return explanatory text along with the code, or the code might not be in a directly usable format. For systematic use, we need more control.

To make the LLM's output reliable for our task, we will:
1. Force a Specific Function Signature: We'll instruct the LLM to define a function with a name and parameters we choose.
2. Parse the Code: We'll extract the Python code from the LLM's response (which is often formatted in Markdown).
3. Verify and Execute: We'll compile the extracted code and then test it against predefined test cases.

In [4]:
PROMPT_WITH_SIGNATURE = """
Problem: You are given a list of integers from 1 to n, with exactly one number missing. 
Write a Python function to find the missing number.

Your solution should we wrapped in a Markdown Python block code. 
```python
def find_missing_number(numbers: list[int]) -> int:
    ...
```
"""

response = client.models.generate_content(model=MODEL, contents=[PROMPT_WITH_SIGNATURE])
print(response.text)

```python
def find_missing_number(numbers: list[int]) -> int:
    """
    Finds the missing number in a list of integers from 1 to n, with exactly one number missing.

    Args:
        numbers: A list of integers from 1 to n, with exactly one number missing.

    Returns:
        The missing number.
    """
    n = len(numbers) + 1
    expected_sum = n * (n + 1) // 2
    actual_sum = sum(numbers)
    return expected_sum - actual_sum
```


We can see that LLM was able to follow our instructions. Right now we need to parse the result:

In [5]:
import ast
import re
from typing import Any, Callable


class FunctionParser:
    @staticmethod
    def parse(
        model_response: str, function_name: str
    ) -> Callable[[list[int]], int] | None:
        function_str = FunctionParser.extract_code(model_response)
        if not function_str:
            print("No function found in response")
            return None

        if not FunctionParser.validate_function_syntax(function_str):
            print("Invalid function syntax")
            return None

        namespace: dict[str, Any] = {}
        # WARNING: This is not safe and should not be used in production
        # This should be run in a sandboxed environment
        exec(function_str, namespace)
        return namespace[function_name]

    @staticmethod
    def validate_function_syntax(function_str: str) -> bool:
        try:
            ast.parse(function_str)
            return True
        except SyntaxError:
            return False

    @staticmethod
    def extract_code(text: str) -> str | None:
        pattern = r"```python\s*(.*?)\s*```"
        match = re.search(pattern, text, re.DOTALL)
        return match.group(1) if match else None


function = FunctionParser.parse(response.text, "find_missing_number")

We can also verify if the result is correct:

In [6]:
assert function([1, 3, 4, 5]) == 2

We can implement automatic verification methods, including those that quantify the quality of our function. A simple yet effective metric is the percentage of test cases passed.

In [7]:
from typing import Generator
import random

class FunctionVerifier:
    def __init__(
        self,
        sizes: list[int] | None = None,
        *,
        rng_seed: int | None = None,
    ) -> None:
        self.sizes = sizes if sizes is not None else [5, 10, 20]
        self._rng = random.Random(rng_seed)

    def _test_cases(self) -> Generator[tuple[list[int], int], None, None]:
        for n in self.sizes:
            full = list(range(1, n + 1))
            for idx in range(n):
                data = full.copy()
                missing = data.pop(idx)
                self._rng.shuffle(data)
                yield data, missing

    def verify(
        self,
        func: Callable[[list[int]], int],
    ) -> float:
        solved = 0
        total = 0

        for data, expected in self._test_cases():
            try:
                result = func(data.copy())
                if result == expected:
                    solved += 1
            except Exception as e:
                print(f"Error during test case execution: {e}")
            total += 1

        return solved / total if total > 0 else 0.0


verifier = FunctionVerifier()
test_pass_rate = verifier.verify(function)
print(f"Test pass rate: {test_pass_rate:.2%}")

Test pass rate: 100.00%


So, we've established that LLMs can generate Python functions, and we can programmatically parse and verify their correctness. This forms the foundation for using LLMs in more complex algorithmic tasks, especially when combined with evolutionary approaches!

### 4. LLM × Evolutionary Algorithms

How can large language models be leveraged for optimization? One promising direction is to employ LLMs to generate novel optimization algorithms in much the same way they are used to synthesize problem-solving functions. Given that algorithms can be expressed as Python functions, this presents a natural and flexible framework for exploring algorithmic design via language models.

In [8]:
PROMPT_METAHEURISTIC = """
Problem: You are tasked with inventing a novel metaheuristic algorithm capable of minimizing an arbitrary real-valued, 
black-box, single-objective function defined over simple bound constraints.

Write a Python function that implements your algorithm. The function must take exactly these arguments:
- function: Callable[np.ndarray], float] - the objective function to minimise.
- bounds: list[tuple[float, float]] - a list of (lower, upper) pairs delimiting the search space for each dimension.
- budget: int - the total number of objective-function evaluations the algorithm may perform.

The function should return a tuple[float, np.ndarray] containing the best objective value found and the corresponding decision vector.

Your solution should be wrapped in a Markdown Python code block.

```python
import numpy as np 

def new_metaheuristic(
	function: Callable[[np.ndarray], float], 
    bounds: list[tuple[float, float]], 
    budget: int
) -> tuple[float, np.ndarray]:
    ...
```
"""

response = client.models.generate_content(model=MODEL, contents=[PROMPT_METAHEURISTIC])
print(response.text)

```python
import numpy as np
from typing import Callable, List, Tuple

def new_metaheuristic(
    function: Callable[[np.ndarray], float],
    bounds: List[Tuple[float, float]],
    budget: int
) -> Tuple[float, np.ndarray]:
    """
    A novel metaheuristic algorithm for minimizing a black-box function 
    subject to bound constraints. This implementation uses a simplified 
    version of differential evolution.

    Args:
        function: The objective function to minimize.
        bounds: A list of (lower, upper) bounds for each dimension.
        budget: The total number of function evaluations allowed.

    Returns:
        A tuple containing the best objective value found and the 
        corresponding decision vector.
    """

    dimension = len(bounds)
    population_size = min(100, budget)  # Limit population size based on budget
    if population_size < 2:
      population_size = 2 #Ensure at least 2 individuals

    # Initialize population within bounds
    population = n

### Exercise 1:
Describe the meta‑heuristic generated by Gemini. Does the idea make sense? Is it novel? What are its weaknesses? 

Let's parse it:

In [9]:
metaheuristic = FunctionParser.parse(response.text, "new_metaheuristic")

And quantify its quality (average across 10 runs for Rastrigin in 10D):

In [13]:
import random
from typing import Callable, Generator

import numpy as np


def rastrigin(x: np.ndarray) -> float:
    A: float = 10.0
    return float(A * len(x) + np.sum(x**2 - A * np.cos(2 * np.pi * x)))


class OptimizerVerifier:
    def __init__(
        self,
        budget: int = 10_000,
        dims: int = 10,
        seeds_count: int = 10,
        test_function: Callable = rastrigin,
    ) -> None:
        self.budget = budget
        self.dims = dims
        self.seeds_count = seeds_count
        self.test_function = test_function

    def verify(
        self,
        optimizer: Callable,
    ) -> dict[str, float]:
        bounds = [(-5, 5) for _ in range(self.dims)]
        results = []
        for seed in range(self.seeds_count):
            np.random.seed(seed)
            random.seed(seed)
            best_val, _ = optimizer(self.test_function, bounds, self.budget)
            results.append(best_val)
        return np.mean(best_val)


verifier = OptimizerVerifier()
mean_score_for_rastrigin = verifier.verify(metaheuristic)
mean_score_for_rastrigin

21.795153194566552

Now let's go one step further. We can calculate the quality (fitness function) for each solution (metaheuristic) generated by LLM. In this case we can try to apply crossover/mutation operators to textual solutions. We can visualize it as optimization process in the space of Python functions that represent different optimization algorithms.

### Exercise 2:
Review [Evolution of Heuristics: Towards Efficient Automatic Algorithm Design Using Large Language Model](https://arxiv.org/pdf/2401.02051) and read **3.4 Prompt Strategies**. Re-implement one of the prompt strategies from **3.4 Prompt Strategies** of Evolution of Heuristics. Generate N = 5 distinct algorithms, benchmark them and report the best.

In [15]:
m1_prompt = f"""
Problem: You are tasked with improving a provided metaheuristic algorithm capable of minimizing an arbitrary real-valued, 
black-box, single-objective function defined over simple bound constraints.
You will be provided a Python function that implements a metaheuristic.
Your task is to improve it's performance.

Python code to improve:
{response.text}


Your solution should be wrapped in a Markdown Python code block.

```python
import numpy as np 

def new_metaheuristic(
	function: Callable[[np.ndarray], float], 
    bounds: list[tuple[float, float]], 
    budget: int
) -> tuple[float, np.ndarray]:
    ...
```
"""


m1_response = client.models.generate_content(model=MODEL, contents=[m1_prompt])
print(m1_response.text)

```python
import numpy as np
from typing import Callable, List, Tuple

def new_metaheuristic(
    function: Callable[[np.ndarray], float],
    bounds: List[Tuple[float, float]],
    budget: int
) -> Tuple[float, np.ndarray]:
    """
    A novel metaheuristic algorithm for minimizing a black-box function 
    subject to bound constraints. This implementation uses a simplified 
    version of differential evolution.

    Args:
        function: The objective function to minimize.
        bounds: A list of (lower, upper) bounds for each dimension.
        budget: The total number of function evaluations allowed.

    Returns:
        A tuple containing the best objective value found and the 
        corresponding decision vector.
    """

    dimension = len(bounds)
    population_size = min(100, budget)  # Limit population size based on budget
    if population_size < 2:
      population_size = 2 #Ensure at least 2 individuals

    # Initialize population within bounds
    population = n

In [ ]:
def e1(parent_funs):
    e1_prompt = f"""
    Problem: You are tasked with creating a novel metaheuristic algorithm capable of minimizing an arbitrary real-valued, 
    black-box, single-objective function defined over simple bound constraints.
    You will be provided Python functions that implement some metaheuristics.
    Your task is create a much different solution than the provided ones.

    Functions:
    {parent_funs[0]}
    {parent_funs[1]}
    {parent_funs[2]}
    {parent_funs[3]}
    {parent_funs[4]}

    Your solution should be wrapped in a Markdown Python code block.

    ```python
    import numpy as np 

    def new_metaheuristic(
        function: Callable[[np.ndarray], float], 
        bounds: list[tuple[float, float]], 
        budget: int
    ) -> tuple[float, np.ndarray]:
        ...
    ```
    """

    return client.models.generate_content(model=MODEL, contents=[e1_prompt]).text

In [18]:
base_metaheuristics = [
    client.models.generate_content(model=MODEL, contents=[PROMPT_METAHEURISTIC]).text
    for _ in range(25)
]
def f(text):
    metaheuristic = FunctionParser.parse(text, "new_metaheuristic")
    verifier = OptimizerVerifier()
    return verifier.verify(metaheuristic)

sorted(base_metaheuristics, key=f)


ValueError: operands could not be broadcast together with shapes (20,10) (10,10) 

In [20]:
len(base_metaheuristics)

25

In [17]:
m1_metaheuristic = FunctionParser.parse(m1_response.text, "new_metaheuristic")

verifier.verify(m1_metaheuristic)

40.98108303588725

Run one of these operators, and try to explain what happened? Are the new mutated solution better than previous one? Calculate the quality of new solution and old ones.

### Exercise 3:
Implement Evolution of Heuristics (or at least part of it). Start simple (pseudocode):
```
current_population = [generate_heuristic()]
for _ in range(10):
   parents = current_population[-p:]
   new_solution = E1(parents) # Read 3.4. Prompt Strategies to better understand E1
   f_new_solution = verify(new_solution)
   current_population.append(new_solution)
```
Save all solutions (best would be to have a separate file for each one). Analyse 5 different ones. Plot quality of solution per epoch. Is this iterative process converging?

### Call to Action
The GECCO 2025 conference is hosting a [competition](https://gecco-2025.sigevo.org/Competition?itemId=5104) on LLM-designed Evolutionary Algorithms. If you’re interested in collaborating and participating as a team, feel free to send me a direct message. Let’s explore the opportunity together.

### 5. Recommended Reading
- [AlphaEvolve: A Gemini-powered coding agent for designing advanced algorithms](https://deepmind.google/discover/blog/alphaevolve-a-gemini-powered-coding-agent-for-designing-advanced-algorithms/)
- Shojaee, Parshin, et al. [LLM-SR: Scientific equation discovery via programming with large language models.](https://arxiv.org/abs/2404.18400)
- Romera-Paredes, Bernardino, et al. [Mathematical discoveries from program search with large language models.](https://www.nature.com/articles/s41586-023-06924-6)
- van Stein, Niki, and Thomas Bäck. [Llamea: A large language model evolutionary algorithm for automatically generating metaheuristics.](https://arxiv.org/abs/2405.20132)
- Liu, Fei, et al. [Evolution of heuristics: Towards efficient automatic algorithm design using large language model.](https://arxiv.org/abs/2401.02051)
- van Stein, Niki, et al. [BLADE: Benchmark suite for LLM-driven Automated Design and Evolution of iterative optimisation heuristics](https://arxiv.org/html/2504.20183v1)
- [OpenEvolve](https://github.com/codelion/openevolve)